In [23]:
import pandas as pd
from datetime import datetime, timedelta
from datetime import datetime
import time
import numpy as np
from math import atan2, sin, cos, sqrt, radians
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt 

In [26]:
#loading data
scores12_13 = pd.read_csv("12_13_raw_scores.csv")
scores12_13 = scores12_13.rename(columns = {"Unnamed: 0":"GAME_DATE_EST"})
scores13_14 = pd.read_csv("13_14_raw_scores.csv", index_col=False)
scores13_14 = scores13_14.rename(columns = {"Unnamed: 0":"GAME_DATE_EST"})
scores14_15 = pd.read_csv("14_15_raw_scores.csv", index_col=False)
scores14_15 = scores14_15.rename(columns = {"Unnamed: 0":"GAME_DATE_EST"})
scores15_16 = pd.read_csv("15_16_raw_scores.csv", index_col=False)
scores16_17 = pd.read_csv("16_17_raw_scores.csv", index_col=False)
scores17_18 = pd.read_csv("17_18_raw_scores.csv", index_col=False)
scores18_19 = pd.read_csv("18_19_raw_scores.csv", index_col=False)
scores18_19 = scores18_19.rename(columns = {"Unnamed: 0":"GAME_DATE_EST"})

In [27]:
#top team away, bottom team home
#splitting data set inorder to get home and away teams 
df_new1 = scores18_19[scores18_19.index%2 == 0]

df_new2 = scores18_19[scores18_19.index%2 != 0]
#changing to df to date time for game date
pd.to_datetime(df_new1["GAME_DATE_EST"])
pd.to_datetime(df_new2["GAME_DATE_EST"])
#merging data sets
new_18_19 = pd.merge(df_new2,df_new1[["GAME_ID","TEAM_ABBREVIATION","TEAM_WINS_LOSSES", "PTS", "FG_PCT", "FT_PCT","FG3_PCT","AST","REB","TOV"]], on = "GAME_ID", how = "left")
new_18_19["OT"] = new_18_19["PTS_OT1"] > 0

#creating boxscore df 
bs_18_19_df = new_18_19.drop(columns = ["TEAM_ID","GAME_SEQUENCE","PTS_QTR1","PTS_QTR2","PTS_QTR3","PTS_QTR4","PTS_OT1","PTS_OT2","PTS_OT3",
                                               "PTS_OT4","PTS_OT5","PTS_OT6","PTS_OT7", "PTS_OT8","PTS_OT9","PTS_OT10","FG_PCT_x", "FT_PCT_x","FG3_PCT_x","AST_x","REB_x","TOV_x",
                                               "FG_PCT_y", "FT_PCT_y","FG3_PCT_y","AST_y","REB_y","TOV_y"])
bs_18_19_df = bs_18_19_df.rename(columns = {"TEAM_CITY_NAME" : "Location_of_Game", "TEAM_ABBREVIATION_x" : "Home_Team","PTS_x" : "Pts_Home","PTS_y" : "Pts_Away",
                                            "TEAM_ABBREVIATION_y" : "Away_Team" })

#creating home win and away win columns
bs_18_19_df["Home_Win"]  = bs_18_19_df["Pts_Home"] > bs_18_19_df["Pts_Away"]
bs_18_19_df["Away_Win"]  = bs_18_19_df["Pts_Home"] < bs_18_19_df["Pts_Away"]

#rearranging box score df 
bs_18_19_df = bs_18_19_df.iloc[:,[0,1,3,2,5,10,4,6,8,11,7,9]]

#create active df? - have the home and away split find average from there (n1 of home and n2 of away)
holder_df = new_18_19.drop(columns = ["TEAM_ID","GAME_SEQUENCE","PTS_QTR1","PTS_QTR2","PTS_QTR3","PTS_QTR4","PTS_OT1","PTS_OT2","PTS_OT3",
                                               "PTS_OT4","PTS_OT5","PTS_OT6","PTS_OT7", "PTS_OT8","PTS_OT9","PTS_OT10"])
run_bs_avg_18_19 = holder_df.iloc[:,[0,1,3,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]]

In [28]:
#takes team data and finds average of specific row - want to add prior to previous data information as well 
def average_of_column(team, column):
    avg1 = df_new1[df_new1['TEAM_ABBREVIATION'] == team][column].mean()
    count1 = df_new1[df_new1['TEAM_ABBREVIATION'] == team][column].count()
    avg2 = df_new2[df_new2['TEAM_ABBREVIATION'] == team][column].mean()
    count2 = df_new2[df_new2['TEAM_ABBREVIATION'] == team][column].count()
    return ((avg1*count1) +(avg2*count2))/(count1 + count2)#takes team data and finds average of specific row can also do up to certain date 

def average_of_column_date(team, date, column):
    avg1 = df_new1[(df_new1['TEAM_ABBREVIATION'] == team) & (df_new1['GAME_DATE_EST'] < date)][column].mean()
    count1 = df_new1[(df_new1['TEAM_ABBREVIATION'] == team) & (df_new1['GAME_DATE_EST'] < date)][column].count()
    avg2 = df_new2[(df_new2['TEAM_ABBREVIATION'] == team) & (df_new2['GAME_DATE_EST'] < date)][column].mean()
    count2 = df_new2[(df_new2['TEAM_ABBREVIATION'] == team) & (df_new2['GAME_DATE_EST'] < date)][column].count()
    return ((avg1*count1) +(avg2*count2))/(count1 + count2)

In [29]:
#function to create running box score average - 1230 rows
stat_list = ['PTS','FG_PCT',"FT_PCT","FG3_PCT","AST", "REB", "TOV"]
stat_list_x = ["PTS_x","FG_PCT_x","FT_PCT_x","FG3_PCT_x","AST_x", "REB_x", "TOV_x"]
stat_list_y = ['PTS_y','FG_PCT_y',"FT_PCT_y","FG3_PCT_y","AST_y", "REB_y", "TOV_y"]

for i in range(1230):
    for z in range(len(stat_list)):
        run_bs_avg_18_19[stat_list_x[z]].iloc[i] = average_of_column_date(holder_df['TEAM_ABBREVIATION_x'].iloc[i], holder_df['GAME_DATE_EST'].iloc[i], stat_list[z])
        run_bs_avg_18_19[stat_list_y[z]].iloc[i] = average_of_column_date(holder_df['TEAM_ABBREVIATION_y'].iloc[i], holder_df['GAME_DATE_EST'].iloc[i], stat_list[z])

run_bs_18_19 = run_bs_avg_18_19.fillna(0)

#run_bs_18_19

C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\3149928658.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_bs_avg_18_19[stat_list_x[z]].iloc[i] = average_of_column_date(holder_df['TEAM_ABBREVIATION_x'].iloc[i], holder_df['GAME_DATE_EST'].iloc[i], stat_list[z])
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\3149928658.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_bs_avg_18_19[stat_list_y[z]].iloc[i] = average_of_column_date(holder_df['TEAM_ABBREVIATION_y'].iloc[i], holder_df['GAME_DATE_EST'].iloc[i], stat_list[z])


In [30]:
#rearranging box score df 
#bs_18_19_df
games_18_19 = bs_18_19_df.iloc[:,[0,1,2,3,7]]
games_18_19  # add an empty columns
games_18_19['Games_in_2_x'] = None
games_18_19['Games_in_5_x'] = None
games_18_19['Games_in_7_x'] = None
games_18_19['Games_in_2_y'] = None
games_18_19['Games_in_5_y'] = None
games_18_19['Games_in_7_y'] = None
games_18_19 = games_18_19.iloc[:,[0,1,2,3,5,6,7,4,8,9,10]]
games_18_19
#need to in order for date subtraction
df_new1['GAME_DATE_EST']= df_new1['GAME_DATE_EST'].astype('datetime64[ns]')
df_new2['GAME_DATE_EST']= df_new2['GAME_DATE_EST'].astype('datetime64[ns]')
holder_df['GAME_DATE_EST'] = holder_df['GAME_DATE_EST'].astype('datetime64[ns]')

C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2206307422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_18_19['Games_in_2_x'] = None
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2206307422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_18_19['Games_in_5_x'] = None
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2206307422.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [31]:
#takes team data and finds average of specific row can also do up to certain date 
def game_count(team, dayz, date):
    count1 = df_new1[(df_new1['GAME_DATE_EST'] > date - timedelta(days=dayz)) & (df_new1['GAME_DATE_EST'] < date) & (df_new1['TEAM_ABBREVIATION'] == team)]['TEAM_CITY_NAME'].count()
    count2 = df_new2[(df_new2['GAME_DATE_EST'] > date - timedelta(days=dayz)) & (df_new2['GAME_DATE_EST'] < date) & (df_new2['TEAM_ABBREVIATION'] == team)]['TEAM_CITY_NAME'].count()
    return count1 + count2 

In [32]:
for i in range(1230):
    games_18_19['Games_in_2_x'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_x'].iloc[i],2, holder_df['GAME_DATE_EST'].iloc[i])
    games_18_19['Games_in_5_x'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_x'].iloc[i],5, holder_df['GAME_DATE_EST'].iloc[i])
    games_18_19['Games_in_7_x'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_x'].iloc[i],7, holder_df['GAME_DATE_EST'].iloc[i])
    games_18_19['Games_in_2_y'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_y'].iloc[i],2, holder_df['GAME_DATE_EST'].iloc[i])
    games_18_19['Games_in_5_y'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_y'].iloc[i],5, holder_df['GAME_DATE_EST'].iloc[i])
    games_18_19['Games_in_7_y'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_y'].iloc[i],7, holder_df['GAME_DATE_EST'].iloc[i])

games_18_19

C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2623045423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_18_19['Games_in_2_x'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_x'].iloc[i],2, holder_df['GAME_DATE_EST'].iloc[i])
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2623045423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_18_19['Games_in_5_x'].iloc[i] = game_count(holder_df['TEAM_ABBREVIATION_x'].iloc[i],5, holder_df['GAME_DATE_EST'].iloc[i])
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2623045423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,GAME_DATE_EST,GAME_ID,Location_of_Game,Home_Team,Games_in_2_x,Games_in_5_x,Games_in_7_x,Away_Team,Games_in_2_y,Games_in_5_y,Games_in_7_y
0,2018-10-16,21800001,Boston,BOS,0,0,0,PHI,0,0,0
1,2018-10-16,21800002,Golden State,GSW,0,0,0,OKC,0,0,0
2,2018-10-17,21800003,Charlotte,CHA,0,0,0,MIL,0,0,0
3,2018-10-17,21800004,Detroit,DET,0,0,0,BKN,0,0,0
4,2018-10-17,21800005,Indiana,IND,0,0,0,MEM,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1225,2019-04-10,21801226,Milwaukee,MIL,0,2,3,OKC,1,2,3
1226,2019-04-10,21801227,San Antonio,SAS,0,1,2,DAL,1,2,3
1227,2019-04-10,21801228,Denver,DEN,1,2,3,MIN,1,2,3
1228,2019-04-10,21801229,LA,LAC,0,1,2,UTA,1,2,3


In [33]:
city_long_lat = pd.read_csv("nba_arena_long_lat.csv")
city_long_lat = city_long_lat.rename(columns = {"ABBREVIATION ":"abbv", "LATITUDE":"lat", "LONGITUDE":"long"})
city_long_lat.head()
#function to convert long/lat coords to distance in kms
def long_lat_dist(lat1, long1, lat2, long2):
    
    #find radians of coords
    rlat1 = radians(lat1)
    rlong1 = radians(long1)
    rlat2 = radians(lat2)
    rlong2 = radians(long2)
        
    #find differnece of lat/long
    dlat = rlat1 - rlat2
    dlong = rlong1 - rlong2
    
    #formula
    a = sin((dlat) / 2)**2 + cos(rlat1) * cos(rlat2) * sin(dlong / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    dist = 6373 * c
    
    return dist

In [34]:
def get_dist(newdf, k):
    home_team = newdf.iloc[[k]].Home_Team.values[0]
    prev_home = newdf.iloc[[k-1]].Home_Team.values[0]
    long1 = city_long_lat.loc[city_long_lat['abbv'] ==  home_team].long.values[0]
    lat1 = city_long_lat.loc[city_long_lat['abbv'] ==  home_team].lat.values[0]
    long2 = city_long_lat.loc[city_long_lat['abbv'] ==  prev_home].long.values[0]
    lat2 = city_long_lat.loc[city_long_lat['abbv'] ==  prev_home].lat.values[0]
    dist = long_lat_dist(lat1, long1, lat2, long2)
    return dist

#return of df with all teams and distances
teams = bs_18_19_df["Home_Team"].unique()

In [35]:
def all_team_dist(box_score):
    teams = box_score["Home_Team"].unique()
    #create a dataframe to store distances by each game id
    distance = pd.DataFrame()   
    distance["GAME_ID"] = box_score["GAME_ID"]
    #go through by each team
    for i in range(len(teams)):
        #create a df of every game the team played
        newdf = box_score[(box_score.Home_Team == teams[i]) | (box_score.Away_Team == teams[i])]
        #go through every game in the new df 
        for k in range(len(newdf["GAME_ID"])):
            #if the team was home
            if newdf.iloc[[k]].Home_Team.values[0] == teams[i]:
            #was it the first game of the season
                if k == 0:
                    distance.loc[distance['GAME_ID'] == newdf.iloc[[k]].GAME_ID.values[0], 'Home_Distance'] = 0
                #else calc distance and place it in distance df    
                else:
                    dist1 = get_dist(newdf, k)
                    distance.loc[distance['GAME_ID'] == newdf.iloc[[k]].GAME_ID.values[0], 'Home_Distance'] = dist1
            #team was away then same process
            else:
                if k == 0:
                    distance.loc[distance['GAME_ID'] == newdf.iloc[[k]].GAME_ID.values[0], 'Away_Distance'] = 0
                else:
                    dist2 = get_dist(newdf, k)
                    distance.loc[distance['GAME_ID'] == newdf.iloc[[k]].GAME_ID.values[0], 'Away_Distance'] = dist2

    return distance

In [148]:
def travel_calc(team, dayz, date, df1, df2):
  sum1 =  df1[(df1['GAME_DATE_EST'] > date - timedelta(days=dayz)) & (df1['GAME_DATE_EST'] <= date) & (df1['TEAM_ABBREVIATION_x'] == team)]['Home_Distance'].sum()
  sum2 =  df2[(df2['GAME_DATE_EST'] > date - timedelta(days=dayz)) & (df2['GAME_DATE_EST'] <= date) & (df2['TEAM_ABBREVIATION_y'] == team)]['Away_Distance'].sum()
  return sum1 + sum2

#def ot_prev_game(team, dayz, date, df1, df2):
#  ot1 =  df1[(df1['GAME_DATE_EST'] > date - timedelta(days=dayz)) & (df1['GAME_DATE_EST'] <= date) & (df1['TEAM_ABBREVIATION'] == team)]['OT']
#  ot2 =  df2[(df2['GAME_DATE_EST'] > date - timedelta(days=dayz)) & (df2['GAME_DATE_EST'] <= date) & (df2['TEAM_ABBREVIATION'] == team)]['OT']
#  return ot1 | ot2

def model_stats(y_test, y_pred):
  print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
  print("Precision: ", metrics.precision_score(y_test, y_pred))
  print("Recall: ", metrics.recall_score(y_test, y_pred))

def get_model_stats(y_test, y_pred):
  return metrics.accuracy_score(y_test, y_pred),metrics.precision_score(y_test, y_pred),metrics.recall_score(y_test, y_pred)


In [149]:
distance_18_19 = all_team_dist(bs_18_19_df)
games_18_19_s = games_18_19.iloc[:,[1,4,5,6,8,9,10]]
df_18_19_1 = pd.merge(run_bs_18_19,games_18_19_s, how='inner', on = 'GAME_ID')
df_18_19 = pd.merge(df_18_19_1, distance_18_19, how='inner', on = 'GAME_ID')
h_dist_18_19 = pd.merge(distance_18_19.iloc[:,[0,1]],df_18_19.iloc[:,[0,1,3]], how='inner', on = 'GAME_ID')
a_dist_18_19 = pd.merge(distance_18_19.iloc[:,[0,2]],df_18_19.iloc[:,[0,1,12]], how='inner', on = 'GAME_ID')
# add an empty columns
df_18_19['Dist_in_2_x'] = None
df_18_19['Dist_in_5_x'] = None
df_18_19['Dist_in_7_x'] = None
df_18_19['Dist_in_2_y'] = None
df_18_19['Dist_in_5_y'] = None
df_18_19['Dist_in_7_y'] = None
#df_18_19['OT_Last_x'] = None
#df_18_19['OT_Last_y'] = None
#needed for subtraction
h_dist_18_19["GAME_DATE_EST"] = h_dist_18_19["GAME_DATE_EST"].astype('datetime64[ns]')
a_dist_18_19["GAME_DATE_EST"] = a_dist_18_19["GAME_DATE_EST"].astype('datetime64[ns]')
df_18_19["GAME_DATE_EST"] = df_18_19["GAME_DATE_EST"].astype('datetime64[ns]')
#Have to do for travel function above
df_new1 = scores18_19[scores18_19.index%2 == 0]
df_new2 = scores18_19[scores18_19.index%2 != 0]
df_new1["OT"] = df_new1["PTS_OT1"] > 0
df_new2["OT"] = df_new2["PTS_OT1"] > 0
df_new1["GAME_DATE_EST"] = df_new1["GAME_DATE_EST"].astype('datetime64[ns]')
df_new2["GAME_DATE_EST"] = df_new2["GAME_DATE_EST"].astype('datetime64[ns]')



C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\4270863127.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new1["OT"] = df_new1["PTS_OT1"] > 0
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\4270863127.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new2["OT"] = df_new2["PTS_OT1"] > 0
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\4270863127.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [150]:
#loop to add all distances to all games
for i in range(1230):
    df_18_19['Dist_in_2_x'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_x'].iloc[i],2, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)
    #df_18_19['OT_Last_x'].iloc[i] = ot_prev_game(df_18_19['TEAM_ABBREVIATION_x'].iloc[i],2, df_18_19['GAME_DATE_EST'].iloc[i],df_new1, df_new2)
    df_18_19['Dist_in_5_x'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_x'].iloc[i],5, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)
    df_18_19['Dist_in_7_x'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_x'].iloc[i],7, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)
    df_18_19['Dist_in_2_y'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_y'].iloc[i],2, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)
    #df_18_19['OT_Last_y'].iloc[i] = ot_prev_game(df_18_19['TEAM_ABBREVIATION_y'].iloc[i],2, df_18_19['GAME_DATE_EST'].iloc[i],df_new1, df_new2)
    df_18_19['Dist_in_5_y'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_y'].iloc[i],5, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)
    df_18_19['Dist_in_7_y'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_y'].iloc[i],7, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)

df_18_19 = pd.merge(df_18_19, bs_18_19_df.iloc[:,[1,5]], how='left', on = 'GAME_ID')

C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2328482557.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_18_19['Dist_in_2_x'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_x'].iloc[i],2, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2328482557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_18_19['Dist_in_5_x'].iloc[i] = travel_calc(df_18_19['TEAM_ABBREVIATION_x'].iloc[i],5, df_18_19['GAME_DATE_EST'].iloc[i],h_dist_18_19, a_dist_18_19)
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2328482557.py:6: SettingWithCopyWarning: 
A v

In [151]:
df_18_19
pd.set_option('display.max_columns',100)
df_18_19_2 = df_18_19
df_18_19_2[["Home_Wins", "Home_Losses"]] = df_18_19_2.TEAM_WINS_LOSSES_x.str.split("-", expand = True)
df_18_19_2

,GAME_DATE_EST,GAME_ID,TEAM_CITY_NAME,TEAM_ABBREVIATION_x,TEAM_WINS_LOSSES_x,PTS_x,FG_PCT_x,FT_PCT_x,FG3_PCT_x,AST_x,REB_x,TOV_x,TEAM_ABBREVIATION_y,TEAM_WINS_LOSSES_y,PTS_y,FG_PCT_y,FT_PCT_y,FG3_PCT_y,AST_y,REB_y,TOV_y,OT,Games_in_2_x,Games_in_5_x,Games_in_7_x,Games_in_2_y,Games_in_5_y,Games_in_7_y,Home_Distance,Away_Distance,Dist_in_2_x,Dist_in_5_x,Dist_in_7_x,Dist_in_2_y,Dist_in_5_y,Dist_in_7_y,Home_Win,Home_Wins,Home_Losses
0,2018-10-16,21800001,Boston,BOS,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,PHI,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0
1,2018-10-16,21800002,Golden State,GSW,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,OKC,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0
2,2018-10-17,21800003,Charlotte,CHA,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MIL,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,0,1
3,2018-10-17,21800004,Detroit,DET,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,BKN,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0
4,2018-10-17,21800005,Indiana,IND,1-0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,MEM,0-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2019-04-10,21801226,Milwaukee,MIL,60-22,118.148148,0.477383,0.772556,0.353580,26.037037,49.691358,13.246914,OKC,49-33,114.320988,0.454481,0.715198,0.350407,23.172840,48.012346,13.666667,False,0,2,3,1,2,3,0.000000,1179.275404,0.0,1116.887139,1250.581409,2296.868117,3414.460829,3414.460829,False,60,22
1226,2019-04-10,21801227,San Antonio,SAS,48-34,111.740741,0.479765,0.819457,0.397457,24.580247,44.617284,11.839506,DAL,33-49,109.049383,0.448160,0.741420,0.342000,23.345679,45.358025,13.703704,False,0,1,2,1,2,3,2019.362731,404.939354,2019.362731,2509.915869,4910.098187,1080.811386,1756.683418,1756.683418,True,48,34
1227,2019-04-10,21801228,Denver,DEN,54-28,110.814815,0.467185,0.765222,0.350284,27.432099,46.308642,12.864198,MIN,36-46,112.691358,0.457556,0.788099,0.354704,24.617284,44.839506,12.592593,False,1,2,3,1,2,3,595.877137,1124.440778,1616.305585,3193.54817,3193.54817,1124.440778,1124.440778,2513.931232,True,54,28
1228,2019-04-10,21801229,LA,LAC,48-34,114.802469,0.471543,0.789728,0.387025,23.901235,45.432099,13.827160,UTA,50-32,111.407407,0.468815,0.736568,0.353938,25.950617,46.222222,14.728395,True,0,1,2,1,2,3,543.758097,935.332592,543.758097,1087.516194,1087.516194,1870.665185,2805.997777,3620.617762,True,48,34


In [152]:
#Creating stats
df_18_19_2[["Home_Wins", "Home_Losses"]] = df_18_19_2.TEAM_WINS_LOSSES_x.str.split("-", expand = True).astype("int")
df_18_19_2['Home_Wins'] = df_18_19_2['Home_Wins'].astype("int")
df_18_19_2['Home_Losses'] = df_18_19_2['Home_Losses'].astype("int")
df_18_19_2[["Away_Wins", "Away_Losses"]] = df_18_19_2.TEAM_WINS_LOSSES_y.str.split("-", expand = True)
df_18_19_2['Away_Wins'] = df_18_19_2['Away_Wins'].astype("int")
df_18_19_2['Away_Losses'] = df_18_19_2['Away_Losses'].astype("int")
df_18_19_2["Home_Win_Perc"] = df_18_19_2['Home_Wins']/(df_18_19_2['Home_Wins'] + df_18_19_2['Home_Losses'])
df_18_19_2["Away_Win_Perc"] = df_18_19_2['Away_Wins']/(df_18_19_2['Away_Wins'] + df_18_19_2['Away_Losses'])
df_18_19_2["Win_Perc_Diff"] = df_18_19_2["Home_Win_Perc"] - df_18_19_2["Away_Win_Perc"]
df_18_19_2["PTS_Diff"] = df_18_19_2["PTS_x"] - df_18_19_2["PTS_y"]
df_18_19_2["AST_Diff"] = df_18_19_2["AST_x"] - df_18_19_2["AST_y"]
df_18_19_2["REB_Diff"] = df_18_19_2["REB_x"] - df_18_19_2["REB_y"]
df_18_19_2["TOV_Diff"] = df_18_19_2["TOV_x"] - df_18_19_2["TOV_y"]


In [154]:
#CREATING A DF WITH BAXTER STAT
df_18_19_model = df_18_19_2.drop(columns=['GAME_ID','GAME_DATE_EST','TEAM_CITY_NAME','TEAM_ABBREVIATION_x','TEAM_WINS_LOSSES_x', 'TEAM_ABBREVIATION_y', 'TEAM_WINS_LOSSES_y','OT'], axis = 1)
df_18_19_model["baxter"] = ((df_18_19_2['Dist_in_2_x'] > df_18_19_2.Dist_in_2_x.quantile(0.975)) | (df_18_19_2['Dist_in_5_x'] > df_18_19_2.Dist_in_5_x.quantile(0.975)) | (df_18_19_2['Dist_in_7_x'] > df_18_19_2.Dist_in_7_x.quantile(0.975)) 
                     | (df_18_19_2['Dist_in_2_y'] > df_18_19_2.Dist_in_2_y.quantile(0.975)) | (df_18_19_2['Dist_in_5_y'] > df_18_19_2.Dist_in_5_y.quantile(0.975)) | (df_18_19_2['Dist_in_7_y'] > df_18_19_2.Dist_in_7_y.quantile(0.975)))
df_18_19_model["baxter_home"] = ((df_18_19_2['Dist_in_2_x'] > df_18_19_2.Dist_in_2_x.quantile(0.975)) | (df_18_19_2['Dist_in_5_x'] > df_18_19_2.Dist_in_5_x.quantile(0.975)) | (df_18_19_2['Dist_in_7_x'] > df_18_19_2.Dist_in_7_x.quantile(0.975))) 
df_18_19_model["baxter_away"] = ((df_18_19_2['Dist_in_2_y'] > df_18_19_2.Dist_in_2_y.quantile(0.975)) | (df_18_19_2['Dist_in_5_y'] > df_18_19_2.Dist_in_5_y.quantile(0.975)) | (df_18_19_2['Dist_in_7_y'] > df_18_19_2.Dist_in_7_y.quantile(0.975)))
df_18_19_model

,PTS_x,FG_PCT_x,FT_PCT_x,FG3_PCT_x,AST_x,REB_x,TOV_x,PTS_y,FG_PCT_y,FT_PCT_y,FG3_PCT_y,AST_y,REB_y,TOV_y,Games_in_2_x,Games_in_5_x,Games_in_7_x,Games_in_2_y,Games_in_5_y,Games_in_7_y,Home_Distance,Away_Distance,Dist_in_2_x,Dist_in_5_x,Dist_in_7_x,Dist_in_2_y,Dist_in_5_y,Dist_in_7_y,Home_Win,Home_Wins,Home_Losses,Away_Wins,Away_Losses,Home_Win_Perc,Away_Win_Perc,Win_Perc_Diff,PTS_Diff,AST_Diff,REB_Diff,TOV_Diff,baxter,baxter_home,baxter_away
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0,0,1,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,False,False,False
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0,0,1,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,False,False,False
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False,0,1,1,0,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,False,False,False
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0,0,1,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,False,False,False
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True,1,0,0,1,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,118.148148,0.477383,0.772556,0.353580,26.037037,49.691358,13.246914,114.320988,0.454481,0.715198,0.350407,23.172840,48.012346,13.666667,0,2,3,1,2,3,0.000000,1179.275404,0.0,1116.887139,1250.581409,2296.868117,3414.460829,3414.460829,False,60,22,49,33,0.731707,0.597561,0.134146,3.827160,2.864198,1.679012,-0.419753,False,False,False
1226,111.740741,0.479765,0.819457,0.397457,24.580247,44.617284,11.839506,109.049383,0.448160,0.741420,0.342000,23.345679,45.358025,13.703704,0,1,2,1,2,3,2019.362731,404.939354,2019.362731,2509.915869,4910.098187,1080.811386,1756.683418,1756.683418,True,48,34,33,49,0.585366,0.402439,0.182927,2.691358,1.234568,-0.740741,-1.864198,False,False,False
1227,110.814815,0.467185,0.765222,0.350284,27.432099,46.308642,12.864198,112.691358,0.457556,0.788099,0.354704,24.617284,44.839506,12.592593,1,2,3,1,2,3,595.877137,1124.440778,1616.305585,3193.54817,3193.54817,1124.440778,1124.440778,2513.931232,True,54,28,36,46,0.658537,0.439024,0.219512,-1.876543,2.814815,1.469136,0.271605,False,False,False
1228,114.802469,0.471543,0.789728,0.387025,23.901235,45.432099,13.827160,111.407407,0.468815,0.736568,0.353938,25.950617,46.222222,14.728395,0,1,2,1,2,3,543.758097,935.332592,543.758097,1087.516194,1087.516194,1870.665185,2805.997777,3620.617762,True,48,34,50,32,0.585366,0.609756,-0.024390,3.395062,-2.049383,-0.790123,-0.901235,False,False,False


In [160]:
#CREATING BAXTER DF
baxter_df_18_19 = df_18_19[(df_18_19_2['Dist_in_2_x'] > df_18_19_2.Dist_in_2_x.quantile(0.975)) | (df_18_19_2['Dist_in_5_x'] > df_18_19_2.Dist_in_5_x.quantile(0.975)) | (df_18_19_2['Dist_in_7_x'] > df_18_19_2.Dist_in_7_x.quantile(0.975)) 
                     | (df_18_19_2['Dist_in_2_y'] > df_18_19_2.Dist_in_2_y.quantile(0.975)) | (df_18_19_2['Dist_in_5_y'] > df_18_19_2.Dist_in_5_y.quantile(0.975)) | (df_18_19_2['Dist_in_7_y'] > df_18_19_2.Dist_in_7_y.quantile(0.975))]

#baxter_df_18_19 = baxter_df_18_19.drop(columns=['GAME_ID','GAME_DATE_EST','TEAM_CITY_NAME','TEAM_ABBREVIATION_x','TEAM_WINS_LOSSES_x', 'TEAM_ABBREVIATION_y', 'TEAM_WINS_LOSSES_y'], axis = 1)
baxter_df_18_19["B2B_x"] = baxter_df_18_19["Games_in_2_x"] == 1
#baxter_df_18_19["OT_B2B_x"] = ((baxter_df_18_19["B2B_x"] == True) & (True in df_18_19_2['OT_Last_x']))
baxter_df_18_19["4_in_5_x"] = baxter_df_18_19["Games_in_5_x"] >= 3
baxter_df_18_19["in_7_x"] = baxter_df_18_19["Games_in_7_x"] >= 4
baxter_df_18_19["B2B_y"] = baxter_df_18_19["Games_in_2_y"] == 1
#baxter_df_18_19["OT_B2B_y"] = (baxter_df_18_19["B2B_y"] == True) & (True in df_18_19_2['OT_Last_y'])
baxter_df_18_19["4_in_5_y"] = baxter_df_18_19["Games_in_5_y"] >= 3
baxter_df_18_19["in_7_y"] = baxter_df_18_19["Games_in_7_y"] >= 4
baxter_df_18_19["home_pts_more"] = baxter_df_18_19["PTS_x"] > baxter_df_18_19["PTS_y"]
baxter_df_18_19

C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2375376544.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baxter_df_18_19["B2B_x"] = baxter_df_18_19["Games_in_2_x"] == 1
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2375376544.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baxter_df_18_19["4_in_5_x"] = baxter_df_18_19["Games_in_5_x"] >= 3
C:\Users\Marcus\AppData\Local\Temp\ipykernel_25368\2375376544.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,GAME_DATE_EST,GAME_ID,TEAM_CITY_NAME,TEAM_ABBREVIATION_x,TEAM_WINS_LOSSES_x,PTS_x,FG_PCT_x,FT_PCT_x,FG3_PCT_x,AST_x,REB_x,TOV_x,TEAM_ABBREVIATION_y,TEAM_WINS_LOSSES_y,PTS_y,FG_PCT_y,FT_PCT_y,FG3_PCT_y,AST_y,REB_y,TOV_y,OT,Games_in_2_x,Games_in_5_x,Games_in_7_x,Games_in_2_y,Games_in_5_y,Games_in_7_y,Home_Distance,Away_Distance,Dist_in_2_x,Dist_in_5_x,Dist_in_7_x,Dist_in_2_y,Dist_in_5_y,Dist_in_7_y,Home_Win,Home_Wins,Home_Losses,Away_Wins,Away_Losses,Home_Win_Perc,Away_Win_Perc,Win_Perc_Diff,PTS_Diff,AST_Diff,REB_Diff,TOV_Diff,B2B_x,4_in_5_x,in_7_x,B2B_y,4_in_5_y,in_7_y,home_pts_more
45,2018-10-22,21800046,Portland,POR,2-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,WAS,1-2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,0,2,2,0,2,2,0.000000,3779.345400,0.0,0.0,0.0,3779.3454,3779.3454,3779.3454,False,2,1,1,2,0.666667,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,False,False,False,False,False,False,False
57,2018-10-24,21800058,San Antonio,SAS,2-2,121.000000,0.455000,0.749000,0.469667,27.666667,46.333333,9.000000,IND,3-2,108.750000,0.487000,0.641500,0.423250,20.500000,45.750000,15.500000,False,0,2,2,0,2,3,1941.314277,1788.319040,1941.314277,6042.942206,6042.942206,1788.31904,3004.928978,3398.192049,False,2,2,3,2,0.500000,0.600000,-0.100000,12.250000,7.166667,0.583333,-6.500000,False,False,False,False,False,False,True
63,2018-10-25,21800064,Orlando,ORL,2-3,100.000000,0.415750,0.700000,0.328000,24.250000,48.250000,12.500000,POR,3-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,1,3,0,1,2,1796.791475,4064.865940,1796.791475,2236.841126,3622.804568,4064.86594,4064.86594,4064.86594,False,2,3,3,1,0.400000,0.750000,-0.350000,100.000000,24.250000,48.250000,12.500000,False,False,False,False,False,False,True
67,2018-10-26,21800068,New York,NYK,1-5,106.400000,0.417800,0.762800,0.347400,20.000000,43.200000,11.400000,GSW,5-1,119.400000,0.509600,0.855200,0.373800,29.800000,45.800000,16.800000,False,0,2,3,0,2,3,1760.607411,4113.640218,1760.607411,4984.314899,4992.034649,4113.640218,5619.374562,6215.251699,False,1,5,5,1,0.166667,0.833333,-0.666667,-13.000000,-9.800000,-2.600000,-5.400000,False,False,False,False,False,False,False
86,2018-10-29,21800087,Indiana,IND,4-3,111.666667,0.521000,0.688833,0.434833,24.000000,41.166667,14.500000,POR,4-2,122.400000,0.469200,0.869000,0.374600,21.000000,51.200000,13.200000,False,0,1,2,0,2,2,423.431769,1651.300202,423.431769,2442.7945,4231.11354,1651.300202,6044.933575,6044.933575,False,4,3,4,2,0.571429,0.666667,-0.095238,-10.733333,3.000000,-10.033333,1.300000,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,2019-04-09,21801215,New Orleans,NOP,33-49,115.592593,0.474938,0.759407,0.344951,27.000000,47.358025,14.543210,GSW,57-24,117.762500,0.491400,0.804625,0.383700,29.450000,46.125000,13.750000,False,0,2,3,0,2,3,3028.904982,3074.494360,3028.904982,6168.872551,6168.872551,3074.49436,3618.252457,4162.010554,False,33,49,57,24,0.402439,0.703704,-0.301265,-2.169907,-2.450000,1.233025,0.793210,False,False,False,False,False,False,False
1218,2019-04-09,21801219,Los Angeles,LAL,37-45,111.901235,0.471580,0.698309,0.331000,25.518519,46.530864,15.172840,POR,52-29,114.525000,0.468337,0.812650,0.360162,23.125000,48.075000,13.337500,False,0,2,3,0,2,3,0.000000,1331.492958,0.0,0.0,1899.57115,1331.492958,4485.97813,6775.166746,False,37,45,52,29,0.451220,0.641975,-0.190756,-2.623765,2.393519,-1.544136,1.835340,False,False,False,False,False,False,False
1220,2019-04-10,21801221,Brooklyn,BKN,42-40,112.234568,0.450519,0.745358,0.352667,23.765432,46.308642,14.432099,MIA,39-43,105.851852,0.452642,0.689469,0.352494,24.308642,46.246914,14.086420,False,0,2,2,1,2,3,1038.683731,1754.123324,1038.683731,2614.854123,2614.854123,3742.271951,4856.872978,7292.

In [238]:
#18-19 creating log regression based on 
baxter_model_df = baxter_df_18_19.drop(columns=['GAME_ID','GAME_DATE_EST','TEAM_CITY_NAME','TEAM_ABBREVIATION_x','TEAM_WINS_LOSSES_x', 'TEAM_ABBREVIATION_y', 'TEAM_WINS_LOSSES_y'], axis = 1)
X = baxter_model_df.loc[:, baxter_model_df.columns != 'Home_Win']
y = baxter_model_df.Home_Win
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)
# instantiate the model (using the default parameters)
logreg_18_19 = LogisticRegression(random_state=100)
# fit the model with data
logreg_18_19.fit(X_train, y_train)
y_pred = logreg_18_19.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

#confusion score + accuracy
print(cnf_matrix)
print(model_stats(y_test, y_pred))
m_scores_18_19 = get_model_stats(y_test, y_pred)
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
X1 = X.drop(['TOV_Diff','AST_Diff','PTS_Diff','REB_Diff','Win_Perc_Diff'],axis = 1)
fit = bestfeatures.fit(X1,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X1.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

[[ 9  3]
 [ 3 15]]
Accuracy:  0.8
Precision:  0.8333333333333334
Recall:  0.8333333333333334
None
            Specs        Score
25    Dist_in_7_x  5681.297554
23    Dist_in_2_x  4858.368572
21  Home_Distance  4748.057705
24    Dist_in_5_x  3961.021015
26    Dist_in_2_y  3559.156139
22  Away_Distance  2488.577949
28    Dist_in_7_y   332.052455
27    Dist_in_5_y   299.277619
32    Away_Losses    95.674308
29      Home_Wins    68.155290


c:\Users\Marcus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [239]:
#second model using best features
X = baxter_model_df[['Dist_in_2_x', 'Dist_in_7_x', 'Dist_in_5_x', 'Home_Distance','Away_Distance', 'Dist_in_2_y', 'Dist_in_7_y', 'Dist_in_5_y', 'Win_Perc_Diff', 'in_7_y']]
y = baxter_model_df.Home_Win
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)

# instantiate the model (using the default parameters)
logreg2_18_19 = LogisticRegression(random_state=100)
# fit the model with data
logreg2_18_19.fit(X_train, y_train)
y_pred2 = logreg2_18_19.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred2)
print(cnf_matrix)
print(model_stats(y_test, y_pred2))
m_scores_18_192 = get_model_stats(y_test, y_pred2)

[[ 9  3]
 [ 2 16]]
Accuracy:  0.8333333333333334
Precision:  0.8421052631578947
Recall:  0.8888888888888888
None


In [68]:
vegas_18_19 = pd.read_csv("vegas_18_19.csv")

In [55]:
#top team away, bottom team home
#splitting data set inorder to get home and away teams 
v_away = vegas_18_19[vegas_18_19.index%2 == 0]
v_home = vegas_18_19[vegas_18_19.index%2 != 0]
#changing to df to date time for game date
pd.to_datetime(v_away["Date"])
pd.to_datetime(v_home["Date"])
#merging data sets
vegas_18_19_2 = pd.merge(v_home,v_away, on = "GameId", how = "left")
#new_18_19["OT"] = new_18_19["PTS_OT1"] > 0
vegas_18_19_2['Location'] = vegas_18_19_2['Team_x']
vegas_18_19_2


,Date_x,Location_x,Team_x,OppTeam_x,TeamId_x,GameId,PercentBet_ML_x,Open_Line_ML_x,Pinnacle_ML_x,5dimes_ML_x,Heritage_ML_x,Bovada_ML_x,Betonline_ML_x,Average_Line_ML_x,Best_Line_ML_x,Worst_Line_ML_x,PercentBet_Spread_x,Open_Line_Spread_x,Open_Odds_Spread_x,Pinnacle_Line_Spread_x,Pinnacle_Odds_Spread_x,5dimes_Line_Spread_x,5dimes_Odds_Spread_x,Heritage_Line_Spread_x,Heritage_Odds_Spread_x,Bovada_Line_Spread_x,Bovada_Odds_Spread_x,Betonline_Line_Spread_x,Betonline_Odds_Spread_x,Average_Line_Spread_x,Average_Odds_Spread_x,Best_Line_Spread_x,Worst_Line_Spread_x,Best_Odds_Spread_x,Worst_Odds_Spread_x,PercentBet_OU_x,Open_Line_OU_x,Open_Odds_OU_x,Pinnacle_Line_OU_x,Pinnacle_Odds_OU_x,5dimes_Line_OU_x,5dimes_Odds_OU_x,Heritage_Line_OU_x,Heritage_Odds_OU_x,Bovada_Line_OU_x,Bovada_Odds_OU_x,Betonline_Line_OU_x,Betonline_Odds_OU_x,Average_Line_OU_x,Average_Odds_OU_x,...,5dimes_ML_y,Heritage_ML_y,Bovada_ML_y,Betonline_ML_y,Average_Line_ML_y,Best_Line_ML_y,Worst_Line_ML_y,PercentBet_Spread_y,Open_Line_Spread_y,Open_Odds_Spread_y,Pinnacle_Line_Spread_y,Pinnacle_Odds_Spread_y,5dimes_Line_Spread_y,5dimes_Odds_Spread_y,Heritage_Line_Spread_y,Heritage_Odds_Spread_y,Bovada_Line_Spread_y,Bovada_Odds_Spread_y,Betonline_Line_Spread_y,Betonline_Odds_Spread_y,Average_Line_Spread_y,Average_Odds_Spread_y,Best_Line_Spread_y,Worst_Line_Spread_y,Best_Odds_Spread_y,Worst_Odds_Spread_y,PercentBet_OU_y,Open_Line_OU_y,Open_Odds_OU_y,Pinnacle_Line_OU_y,Pinnacle_Odds_OU_y,5dimes_Line_OU_y,5dimes_Odds_OU_y,Heritage_Line_OU_y,Heritage_Odds_OU_y,Bovada_Line_OU_y,Bovada_Odds_OU_y,Betonline_Line_OU_y,Betonline_Odds_OU_y,Average_Line_OU_y,Average_Odds_OU_y,Best_Line_OU_y,Worst_Line_OU_y,Best_Odds_OU_y,Worst_Odds_OU_y,Pts_y,Spread_y,Result_y,Total_y,Location
0,2018-10-16,home,Boston,Philadelphia,1610612738,21800001,54.0,-205.0,-202.0,-200.0,-210.0,-230.0,-210.0,-210.4,-200.0,-230.0,54.9,NaN,NaN,-4.5,-109.0,-4.5,-107.0,-4.5,-105.0,-5.5,-110.0,-4.5,-110.0,-4.70,-108.20,-4.5,-5.5,-105.0,-110.0,43.1,208.5,-110.0,211.0,-105.0,211.0,-108.0,211.5,-105.0,212.0,-115.0,211.5,-110.0,210.92,-108.83,...,185.0,180.0,190.0,180.0,183.2,190.0,180.0,45.1,NaN,NaN,4.5,-101.0,4.5,-103.0,4.5,-105.0,5.5,-110.0,4.5,-110.0,4.70,-105.80,5.5,4.5,-101.0,-110.0,56.9,208.5,-110.0,211.0,-105.0,211.0,-102.0,211.5,-105.0,212.0,-105.0,211.5,-110.0,210.92,-106.17,212.0,208.5,-102.0,-110.0,87,-18,L,192,Boston
1,2018-10-16,home,Golden State,Oklahoma City,1610612744,21800002,44.3,-1000.0,-1002.0,-925.0,-850.0,-1100.0,-1200.0,-1015.4,-850.0,-1200.0,57.6,-11.5,-110.0,-12.5,-113.0,-12.5,-106.0,-12.5,-105.0,-13.5,-110.0,-12.5,-115.0,-12.50,-109.83,-11.5,-13.5,-105.0,-115.0,47.8,223.5,-110.0,220.5,-105.0,220.0,-105.0,220.5,-105.0,220.0,-105.0,220.5,-110.0,220.83,-106.67,...,725.0,600.0,650.0,825.0,713.6,825.0,600.0,42.4,11.5,-110.0,12.5,-102.0,12.5,-104.0,12.5,-105.0,13.5,-110.0,12.5,-105.0,12.50,-106.00,13.5,11.5,-102.0,-110.0,52.2,223.5,-110.0,220.5,-105.0,220.0,-105.0,220.5,-105.0,220.0,-115.0,220.5,-110.0,220.83,-108.33,223.5,220.0,-105.0,-115.0,100,-8,L,208,Golden State
2,2018-10-17,home,Charlotte,Milwaukee,1610612766,21800003,41.1,110.0,145.0,145.0,140.0,150.0,145.0,145.0,150.0,140.0,41.4,NaN,NaN,3.5,-105.0,3.5,-103.0,3.5,-105.0,4.0,-110.0,3.0,-100.0,3.50,-104.60,4.0,3.0,-100.0,-110.0,43.3,217.0,-110.0,222.0,-106.0,222.0,-102.0,222.5,-105.0,221.0,-105.0,222.0,-110.0,221.08,-106.33,...,-155.0,-160.0,-170.0,-165.0,-162.0,-155.0,-170.0,58.6,NaN,NaN,-3.5,-105.0,-3.5,-107.0,-3.5,-105.0,-4.0,-110.0,-3.0,-120.0,-3.50,-109.40,-3.0,-4.0,-105.0,-120.0,56.7,217.0,-110.0,222.0,-104.0,222.0,-108.0,222.5,-105.0,221.0,-115.0,222.0,-110.0,221.08,-108.67,222.5,217.0,-104.0,-115.0,113,1,W,225,Charlotte
3,2018-10-17,home,Detroit,Brooklyn,1610612765,21800004,52.3,NaN,-242.0,-220.0,-250.0,-255.0,-245.0,-242.4,-220.0,-255.0,50.7,-4.5,-110.0,-6.0,-106.0,-6.0,-102.0,-6.0,-105.0,-6.5,-110.0,-6.0,-105.0,-5.83,-106.33,-4.5,-6.5,-102.0,-110.0,53.0,212.0,-110.0,212.5,-107.0,212.5,-105.0,212.5,-105.0,212.0,-105.0,212.5,-110.0,212.33

In [193]:
vegas = vegas_18_19_2.filter(['Date_x','GameId', 'Location', 'Team_x', 'Average_Line_ML_x', 'Average_Line_Spread_x', 'Team_y','Average_Line_ML_y', 'Average_Line_Spread_y'], axis=1)
vegas = vegas.rename(columns={"GameId": "GAME_ID"})
vegas2 = vegas.filter(['GAME_ID', 'Average_Line_ML_x', 'Average_Line_Spread_x', 'Average_Line_ML_y', 'Average_Line_Spread_y'], axis=1)
vegas_baxter = pd.merge(baxter_df_18_19, vegas2, on ='GAME_ID')
vegas2

,GAME_ID,Average_Line_ML_x,Average_Line_Spread_x,Average_Line_ML_y,Average_Line_Spread_y
0,21800001,-210.4,-4.70,183.2,4.70
1,21800002,-1015.4,-12.50,713.6,12.50
2,21800003,145.0,3.50,-162.0,-3.50
3,21800004,-242.4,-5.83,209.2,5.83
4,21800005,-294.4,-7.42,250.2,7.42
...,...,...,...,...,...
1221,21801226,176.2,4.08,-202.4,-4.08
1222,21801227,-1238.6,-14.00,858.2,14.00
1223,21801228,-888.8,-12.67,646.8,12.67
1224,21801229,-311.2,-7.33,262.8,7.33


In [282]:
#vegas_baxter[['Average_Line_ML_x', 'Average_Line_ML_y']]
y_test_df = y_test.to_frame()
y_pred_df = pd.DataFrame(y_pred2, columns = ['Predicted'])
z_df = y_test_df.join(vegas2[['Average_Line_ML_x', 'Average_Line_ML_y']])
z_df["Predicted"] = y_pred2

money_df2 = z_df[['Home_Win','Predicted','Average_Line_ML_x', 'Average_Line_ML_y']]
display(money_df2)
money_df = money_df2.reset_index().drop(['index'], axis=1)
money_df

,Home_Win,Predicted,Average_Line_ML_x,Average_Line_ML_y
97,False,False,-190.60,168.60
210,True,False,206.25,-242.50
225,True,True,-714.60,538.60
1046,True,True,192.60,-222.20
1218,False,False,331.60,-404.20
400,True,True,-241.20,208.00
334,False,False,105.25,-120.50
203,True,True,-980.20,707.40
971,True,True,-161.00,143.80
223,False,False,162.25,-180.75


,Home_Win,Predicted,Average_Line_ML_x,Average_Line_ML_y
0,False,False,-190.60,168.60
1,True,False,206.25,-242.50
2,True,True,-714.60,538.60
3,True,True,192.60,-222.20
4,False,False,331.60,-404.20
5,True,True,-241.20,208.00
6,False,False,105.25,-120.50
7,True,True,-980.20,707.40
8,True,True,-161.00,143.80
9,False,False,162.25,-180.75


In [281]:

total = 0
for i in range(len(money_df)):
    if(money_df["Predicted"][i] == money_df["Home_Win"][i]):
        if(money_df["Predicted"][i]):
            if(int(money_df['Average_Line_ML_x'][i])>0):
                #print(i)
                #print(int(money_df['Average_Line_ML_x'][i]) - 100)
                total += int(money_df['Average_Line_ML_x'][i]) - 100 
            else: 
                #print(i)
                #print(100/abs((int(money_df['Average_Line_ML_x'][i])/100)))
                total += 100/abs((int(money_df['Average_Line_ML_x'][i])/100))
        else: 
            if(int(money_df['Average_Line_ML_y'][i])>0):
                total += int(money_df['Average_Line_ML_y'][i]) - 100 
            else: 
                total += 100/abs((int(money_df['Average_Line_ML_y'][i])/100))
    else: 
        total -= 100
    

print(total)
money_df


1478.409123610142


,Home_Win,Predicted,Average_Line_ML_x,Average_Line_ML_y
0,False,False,-190.60,168.60
1,True,False,206.25,-242.50
2,True,True,-714.60,538.60
3,True,True,192.60,-222.20
4,False,False,331.60,-404.20
5,True,True,-241.20,208.00
6,False,False,105.25,-120.50
7,True,True,-980.20,707.40
8,True,True,-161.00,143.80
9,False,False,162.25,-180.75


In [271]:
#second model using best features
X = baxter_model_df[['Dist_in_2_x', 'Dist_in_7_x', 'Dist_in_5_x', 'Home_Distance','Away_Distance', 'Dist_in_2_y', 'Dist_in_7_y', 'Dist_in_5_y', 'Win_Perc_Diff', 'in_7_y']]
y = baxter_model_df.Home_Win
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)
# instantiate the model (using the default parameters)
money_reg = LogisticRegression(random_state=100)
# fit the model with data
money_reg.fit(X, y)
y_pred_money = money_reg.predict(X)
#cnf_matrix = metrics.confusion_matrix(y_test, y_pred2)
#print(cnf_matrix)
#print(model_stats(y_test, y_pred2))
#m_scores_18_192 = get_model_stats(y_test, y_pred2)
#vegas_baxter[['Average_Line_ML_x', 'Average_Line_ML_y']]
y_df = y.to_frame()
z2_df = y_df.join(vegas2[['Average_Line_ML_x', 'Average_Line_ML_y']])
z2_df["Predicted"] = y_pred_money

money_all_df2 = z2_df[['Home_Win','Predicted','Average_Line_ML_x', 'Average_Line_ML_y']]
money_all_df2 = money_all_df2.reset_index().drop(['index'], axis=1)
money_all_df2

,Home_Win,Predicted,Average_Line_ML_x,Average_Line_ML_y
0,False,False,-191.6,168.8
1,False,True,-122.6,106.6
2,False,False,141.8,-158.8
3,False,False,674.0,-935.2
4,False,True,-168.4,151.0
...,...,...,...,...
142,False,True,386.0,-483.4
143,False,True,331.6,-404.2
144,True,False,175.4,-200.6
145,True,True,-888.8,646.8


In [278]:
total2= 0
for i in range(len(money_all_df2)):
    if(money_all_df2["Predicted"][i] == money_all_df2["Home_Win"][i]):
        if(money_all_df2["Predicted"][i]):
            if(int(money_all_df2['Average_Line_ML_x'][i])>0):
                #print(i)
                #print(int(money_all_df2['Average_Line_ML_x'][i]) - 100)
                total2+= int(money_all_df2['Average_Line_ML_x'][i]) - 100 
            else: 
                #print(i)
                #print(100/abs((int(money_all_df2['Average_Line_ML_x'][i])/100)))
                total2+= 100/abs((int(money_all_df2['Average_Line_ML_x'][i])/100))
        else: 
            if(int(money_all_df2['Average_Line_ML_y'][i])>0):
                total2+= int(money_all_df2['Average_Line_ML_y'][i]) - 100 
            else: 
                total2+= 100/abs((int(money_all_df2['Average_Line_ML_y'][i])/100))
    else: 
        total2-= 100
    

print(total2)
display(money_all_df2)

8305.107373072555


,Home_Win,Predicted,Average_Line_ML_x,Average_Line_ML_y
0,False,False,-191.6,168.8
1,False,True,-122.6,106.6
2,False,True,141.8,-158.8
3,False,False,674.0,-935.2
4,False,False,-168.4,151.0
...,...,...,...,...
142,False,False,386.0,-483.4
143,False,False,331.6,-404.2
144,True,True,175.4,-200.6
145,True,True,-888.8,646.8


In [277]:
#NO TRAIN AND SPLIT ALL FEATURES
X = baxter_model_df.loc[:, baxter_model_df.columns != 'Home_Win']
y = baxter_model_df.Home_Win
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)
# instantiate the model (using the default parameters)
money_reg = LogisticRegression(random_state=100)
# fit the model with data
money_reg.fit(X, y)
y_pred_money = money_reg.predict(X)
#cnf_matrix = metrics.confusion_matrix(y_test, y_pred2)
#print(cnf_matrix)
#print(model_stats(y_test, y_pred2))
#m_scores_18_192 = get_model_stats(y_test, y_pred2)
#vegas_baxter[['Average_Line_ML_x', 'Average_Line_ML_y']]
y_df = y.to_frame()
z2_df = y_df.join(vegas2[['Average_Line_ML_x', 'Average_Line_ML_y']])
z2_df["Predicted"] = y_pred_money

money_all_df2 = z2_df[['Home_Win','Predicted','Average_Line_ML_x', 'Average_Line_ML_y']]
money_all_df2 = money_all_df2.reset_index().drop(['index'], axis=1)
display(money_all_df2)

total2= 0
for i in range(len(money_all_df2)):
    if(money_all_df2["Predicted"][i] == money_all_df2["Home_Win"][i]):
        if(money_all_df2["Predicted"][i]):
            if(int(money_all_df2['Average_Line_ML_x'][i])>0):
                #print(i)
                #print(int(money_all_df2['Average_Line_ML_x'][i]) - 100)
                total2+= int(money_all_df2['Average_Line_ML_x'][i]) - 100 
            else: 
                #print(i)
                #print(100/abs((int(money_all_df2['Average_Line_ML_x'][i])/100)))
                total2+= 100/abs((int(money_all_df2['Average_Line_ML_x'][i])/100))
        else: 
            if(int(money_all_df2['Average_Line_ML_y'][i])>0):
                total2+= int(money_all_df2['Average_Line_ML_y'][i]) - 100 
            else: 
                total2+= 100/abs((int(money_all_df2['Average_Line_ML_y'][i])/100))
    else: 
        total2-= 100
    

print(total2)

c:\Users\Marcus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Home_Win,Predicted,Average_Line_ML_x,Average_Line_ML_y
0,False,False,-191.6,168.8
1,False,True,-122.6,106.6
2,False,True,141.8,-158.8
3,False,False,674.0,-935.2
4,False,False,-168.4,151.0
...,...,...,...,...
142,False,False,386.0,-483.4
143,False,False,331.6,-404.2
144,True,True,175.4,-200.6
145,True,True,-888.8,646.8


8305.107373072555
